1. Install Python Libraries: Ensure you have osmnx, geopandas, and geojson installed
pip install osmnx geopandas geojson

2. Extract OSM Road Network with osmnx
Since your project uses osmnx, let’s fetch Calgary’s road network and export it as GeoJSON (compatible with Mapbox/Leaflet).

In [24]:
import osmnx as ox  # type: ignore
import geopandas as gpd

In [14]:
# Define the place (Calgary)
place_name = "Calgary, Alberta, Canada"

# Fetch the road network (drivable roads)
G = ox.graph_from_place(place_name, network_type="drive")

# Convert the graph to GeoDataFrames (nodes and edges)
nodes, edges = ox.graph_to_gdfs(G)


In [15]:
#Before saving
print("Available columns in edges:", edges.columns.tolist())
print(f"Number of edges in G: {len(G)}")
print(f"Number of rows in edges GeoDataFrame: {len(edges)}")

Available columns in edges: ['osmid', 'highway', 'lanes', 'maxspeed', 'name', 'oneway', 'ref', 'reversed', 'length', 'geometry', 'width', 'bridge', 'tunnel', 'junction', 'access']
Number of edges in G: 36044
Number of rows in edges GeoDataFrame: 84931


In [ ]:
# Save edges as GeoJSON (this will be your road network)
edges.to_file("calgary_roads.geojson", driver="GeoJSON")

print("Calgary road network saved as calgary_roads.geojson")

In [3]:
# Inspect available columns
print("Available columns in edges:", edges.columns.tolist())

Available columns in edges: ['osmid', 'highway', 'lanes', 'maxspeed', 'name', 'oneway', 'ref', 'reversed', 'length', 'geometry', 'width', 'bridge', 'tunnel', 'junction', 'access']


Dataset exploration for analysing the max speed feature 

In [9]:
# Total number of edges (rows)
total_edges = len(G)
print(f"Total number of edges: {total_edges}")

Total number of edges: 36044


In [12]:
gdf = gpd.read_file('calgary_roads.geojson')

# Display the available columns
print("Available columns:", gdf.columns.tolist())

# Total number of edges (rows)
total_edges = len(gdf)
print(f"Total number of edges: {total_edges}")

Skipping field highway: unsupported OGR type: 5
Skipping field lanes: unsupported OGR type: 5
Skipping field maxspeed: unsupported OGR type: 5
Skipping field name: unsupported OGR type: 5
Skipping field access: unsupported OGR type: 5


Available columns: ['u', 'v', 'key', 'osmid', 'oneway', 'ref', 'reversed', 'length', 'width', 'bridge', 'tunnel', 'junction', 'geometry']
Total number of edges: 84931


In [10]:
# --- Analyze missing 'maxspeed' values overall ---
# Count the number of edges with missing 'maxspeed'
missing_maxspeed = G['maxspeed'].isna().sum()
percent_missing_maxspeed = (missing_maxspeed / total_edges) * 100
print(f"Number of edges missing 'maxspeed': {missing_maxspeed}")
print(f"Percentage of edges missing 'maxspeed': {percent_missing_maxspeed:.2f}%")

KeyError: 'maxspeed'

In [5]:
# Define the place
place_name = "Calgary, Alberta, Canada"

# Fetch the road network
print("Downloading road network...")
G = ox.graph_from_place(place_name, network_type="drive")

In [9]:
# Add edge speeds and travel times
hwy_speeds = {
    'motorway': 100,
    'trunk': 80,
    'primary': 60,
    'secondary': 60,
    'tertiary': 50,
    'residential': 40,
    'motorway_link': 60,
    'primary_link': 50,
    'secondary_link': 50,
    'tertiary_link': 40
}
G = ox.add_edge_speeds(G, hwy_speeds=hwy_speeds)
G = ox.add_edge_travel_times(G)
print("Added speeds and travel times to the graph")

# Save the graph as GraphML
ox.save_graphml(G, filepath="calgary_roads.graphml")
print("Graph saved as calgary_roads.graphml")

# Convert the graph to GeoDataFrames
nodes, edges = ox.graph_to_gdfs(G)

# Add the 'time' column to the edges GeoDataFrame (travel_time in seconds)
edges['time'] = edges['travel_time']



Added speeds and travel times to the graph
Graph saved as calgary_roads.graphml


In [10]:
# Preprocess columns to handle lists
for col in ['osmid', 'highway', 'lanes', 'maxspeed', 'name', 'access', 'ref']:
    edges[col] = edges[col].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)

# Save the updated edges as GeoJSON
edges.to_file("calgary_roads.geojson", driver="GeoJSON")
print("Updated calgary_roads.geojson with 'time' column")
print("Available columns in edges:", edges.columns.tolist())
print(f"Number of edges in G: {len(G)}")
print(f"Number of rows in edges GeoDataFrame: {len(edges)}")

Updated calgary_roads.geojson with 'time' column
Available columns in edges: ['osmid', 'highway', 'lanes', 'maxspeed', 'name', 'oneway', 'ref', 'reversed', 'length', 'geometry', 'speed_kph', 'travel_time', 'width', 'bridge', 'tunnel', 'junction', 'access', 'time']
Number of edges in G: 36044
Number of rows in edges GeoDataFrame: 84931


3. regenerate calgary_roads.geojson with the required properties: "incident_risk" (e.g., from DBSCAN) 
Next: Update incident_risk with DBSCAN clustering logic

In [5]:
# Ensure osmid and name are present (osmnx should include these)
# Add incident_risk if missing (dummy data for now)
if 'incident_risk' not in edges.columns:
    edges['incident_risk'] = 0.0  # Replace with your DBSCAN logic later
    print("Added incident_risk column with default value 0.0")

Added incident_risk column with default value 0.0


In [6]:
# Save to GeoJSON
edges.to_file("calgary_roads.geojson", driver="GeoJSON")
print("Saved calgary_roads.geojson with properties")

Saved calgary_roads.geojson with properties


In [7]:
# Inspect available columns
print("Available columns in edges:", edges.columns.tolist())

Available columns in edges: ['osmid', 'highway', 'lanes', 'maxspeed', 'name', 'oneway', 'ref', 'reversed', 'length', 'geometry', 'width', 'bridge', 'tunnel', 'junction', 'access', 'incident_risk']


In [8]:
# Save the graph as GraphML (or another format) to use it locally
ox.save_graphml(G, filepath="calgary_roads.graphml")
print("Graph saved as calgary_roads.graphml")

Graph saved as calgary_roads.graphml


In [12]:
import networkx as nx

print("NetworkX version:", nx.__version__)
try:
    print("k_shortest_paths available:", hasattr(nx, 'k_shortest_paths'))
    # Create a simple MultiDiGraph
    G = nx.MultiDiGraph()
    G.add_edge(0, 1, 0, weight=1)
    G.add_edge(1, 2, 0, weight=1)
    paths = list(nx.k_shortest_paths(G, 0, 2, k=1, weight='weight'))
    print("Test path:", paths)
except AttributeError as e:
    print("Error:", str(e))

NetworkX version: 3.4.2
k_shortest_paths available: False
Error: module 'networkx' has no attribute 'k_shortest_paths'


In [1]:
# test_networkx.py
import networkx as nx

print("NetworkX version:", nx.__version__)
try:
    print("k_shortest_paths available:", hasattr(nx, 'k_shortest_paths'))
    # Create a simple MultiDiGraph
    G = nx.MultiDiGraph()
    G.add_edge(0, 1, 0, weight=1)
    G.add_edge(1, 2, 0, weight=1)
    paths = list(nx.k_shortest_paths(G, 0, 2, k=1, weight='weight'))
    print("Test path:", paths)
except AttributeError as e:
    print("Error:", str(e))

NetworkX version: 3.4.2
k_shortest_paths available: False
Error: module 'networkx' has no attribute 'k_shortest_paths'
